In [1]:
##### This Jupyter notebook generates FLDA CVs. 
##### Please ensure you have properly set up the conda environment with all libraries.
##### Please note that this code could be simplified using the scikit-learn library. However, it was written from scratch.

##### Author: MO (latest update: May 28, 2024)

In [2]:
##### User Inputs #####
nDataPoints = 754 # Number of data points in each class (*note: each class should have the same number of data points)
num_class = 3 # Number of classes
num_descriptor = 7 # Number of descriptors or features
num_eigenvector = 2 # Number of eigenvectors or CVs (reduced dimensionality)
descriptor_list = ['res159.439', 'res245.369', 'res64.137', 'res199.471', 'res78.450', 'res242.340', 'res77.293'] # List of feature names

In [3]:
### STEP 0. Import libraries
import pandas as pd
import numpy as np

In [4]:
### STEP 1. Load input data
df = pd.read_csv('mpso.csv')

In [5]:
### STEP 2. Zero-mean the data
np.set_printoptions(precision=8)
for elem in descriptor_list:
    print('Mean for ', elem, ': ', df[elem].mean())
    df[elem] = df[elem] - df[elem].mean()

Mean for  res159.439 :  21.74415728372237
Mean for  res245.369 :  38.09927344891246
Mean for  res64.137 :  16.775558394920427
Mean for  res199.471 :  30.131755684036246
Mean for  res78.450 :  13.889633035910256
Mean for  res242.340 :  26.819251453872678
Mean for  res77.293 :  40.03164760639258


In [6]:
### STEP 3. Separate data and generate labels
X = df.iloc[:,:num_descriptor].values
X = X.astype(np.float64)
y = np.concatenate([np.zeros(nDataPoints)+1,np.ones(nDataPoints)+1,np.ones(nDataPoints)+2])
print(X)
print(y)

[[-0.76525385  2.08614583  1.03468783 ...  0.23212689  1.08992405
   0.75515021]
 [-1.47115657  2.44363145  0.69819475 ...  2.78658885  1.9426788
   2.18380523]
 [-2.18411414  2.18471138 -0.08038096 ...  1.60488209  1.93621906
   1.18560777]
 ...
 [ 2.30423513 -2.98455501  2.50959053 ... -3.00694806 -2.71507559
  -1.41571489]
 [ 3.33669351 -3.31498275  2.7419514  ... -3.27287224 -3.22628615
  -1.51252031]
 [ 1.98100839 -3.30819255  2.2219749  ... -2.74602217 -3.0444609
  -1.75634772]]
[1. 1. 1. ... 3. 3. 3.]


In [7]:
### STEP 4. Compute the d-dimensional mean vectors
### Here, we calculate #num_class column vectors, each of which contains #num_descriptor elements (means)
np.set_printoptions(precision=4)
mean_vectors = []
for cl in range(1, num_class+1):
    mean_vectors.append(np.mean(X[y==cl], axis=0)) 
    print(f'Mean Vector class {cl}: {mean_vectors[cl-1]}')

Mean Vector class 1: [-0.522   1.7285  0.7063  2.3102  3.008   1.4236  0.4796]
Mean Vector class 2: [-1.0786  1.446  -3.2836 -0.1895  0.5859  0.4779  2.1154]
Mean Vector class 3: [ 1.6007 -3.1745  2.5773 -2.1207 -3.5939 -1.9015 -2.595 ]


In [8]:
### STEP 5. Compute the scatter matrices
### 5-1. Within-class scatter matrix SW
S_W = np.zeros((num_descriptor,num_descriptor))
for cl, mv in zip(range(1, num_class+1), mean_vectors):
    class_sc_mat = np.zeros((num_descriptor,num_descriptor))
    for row in X[y==cl]:
        row, mv = row.reshape(num_descriptor,1), mv.reshape(num_descriptor,1)
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat  

print('within-class Scatter Matrix:\n', S_W)

within-class Scatter Matrix:
 [[1740.0633 -144.7306  513.0246  -61.5244 1005.4498 -216.4135 -178.9632]
 [-144.7306 1170.8592 -184.3044  692.1579  394.8336  492.9423  236.9878]
 [ 513.0246 -184.3044 2951.7439 -150.3187  535.1183  100.5744    8.8308]
 [ -61.5244  692.1579 -150.3187 1968.6852  276.1215  338.1028  337.5499]
 [1005.4498  394.8336  535.1183  276.1215 2890.0727  108.4058  291.0696]
 [-216.4135  492.9423  100.5744  338.1028  108.4058  905.9528  139.249 ]
 [-178.9632  236.9878    8.8308  337.5499  291.0696  139.249  1566.1371]]


In [9]:
### 5-2. Between-class scatter matrix SB
overall_mean = np.mean(X, axis=0) 
S_B = np.zeros((num_descriptor,num_descriptor))
for i, mean_vec in enumerate(mean_vectors):
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(num_descriptor,1)
    overall_mean = overall_mean.reshape(num_descriptor,1)
    S_B += n*(mean_vec-overall_mean).dot((mean_vec-overall_mean).T)

print('between-class Scatter Matrix:\n', S_B)

between-class Scatter Matrix:
 [[  3014.6095  -5687.7397   5503.1252  -3314.7076  -5997.9838  -3244.0209
   -5041.2023]
 [ -5687.7397  11427.8272  -8828.4601   7880.4565  13161.463    6927.9563
    9142.848 ]
 [  5503.1252  -8828.4601  13514.3623  -2421.6046  -6832.5517  -4120.3529
  -10024.9322]
 [ -3314.7076   7880.4565  -2421.6046   7442.3423  10902.6679   5452.0663
    4682.6165]
 [ -5997.9838  13161.463   -6832.5517  10902.6679  16819.8248   8592.6645
    9054.2095]
 [ -3244.0209   6927.9563  -4120.3529   5452.0663   8592.6645   4426.6344
    4997.7325]
 [ -5041.2023   9142.848  -10024.9322   4682.6165   9054.2095   4997.7325
    8625.1312]]


In [12]:
### STEP 6. Solve the generalized eigenvalue problem for the matrix SW^-1.SB
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
for i in range(len(eig_vals)):
    eigvec_sc = eig_vecs[:,i].reshape(num_descriptor,1)
    print(f'\nEigenvector {i+1}: \n{eigvec_sc.real}')
    print(f'Eigenvalue {i+1}: {eig_vals[i].real:.2e}')

for i in range(len(eig_vals)): # Check the eigenvector-eigenvalue calculation
    eigv = eig_vecs[:,i].reshape(num_descriptor,1)
    np.testing.assert_array_almost_equal(np.linalg.inv(S_W).dot(S_B).dot(eigv), eig_vals[i] * eigv, decimal=3, err_msg='', verbose=True)
print('Good!')


Eigenvector 1: 
[[-0.42  ]
 [ 0.4925]
 [-0.3204]
 [-0.028 ]
 [ 0.522 ]
 [ 0.2858]
 [ 0.3515]]
Eigenvalue 1: 2.05e+01

Eigenvector 2: 
[[ 0.9602]
 [ 0.158 ]
 [-0.1247]
 [-0.0611]
 [ 0.1056]
 [ 0.0504]
 [ 0.1419]]
Eigenvalue 2: -2.46e-15

Eigenvector 3: 
[[-0.2525]
 [-0.2693]
 [ 0.396 ]
 [ 0.5397]
 [ 0.3364]
 [ 0.2591]
 [-0.485 ]]
Eigenvalue 3: 5.40e+00

Eigenvector 4: 
[[ 0.4855]
 [ 0.6127]
 [ 0.0465]
 [-0.2731]
 [-0.1957]
 [ 0.3151]
 [-0.1406]]
Eigenvalue 4: 4.39e-17

Eigenvector 5: 
[[ 0.4855]
 [ 0.6127]
 [ 0.0465]
 [-0.2731]
 [-0.1957]
 [ 0.3151]
 [-0.1406]]
Eigenvalue 5: 4.39e-17

Eigenvector 6: 
[[ 0.1132]
 [ 0.0341]
 [-0.2866]
 [ 0.628 ]
 [-0.5422]
 [ 0.3653]
 [-0.2866]]
Eigenvalue 6: -2.94e-16

Eigenvector 7: 
[[ 0.5514]
 [ 0.4051]
 [-0.0499]
 [-0.08  ]
 [-0.3396]
 [ 0.6255]
 [-0.1276]]
Eigenvalue 7: 5.93e-17
Good!


In [13]:
### STEP 7. Select linear discriminants for the new feature subspace
### 7-1. Sort the eigenvectors by decreasing eigenvalues
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)

print('Eigenvalues in decreasing order:\n')
for i in eig_pairs:
    print(i[0])

print('Variance explained:\n')
eigv_sum = sum(eig_vals)
for i,j in enumerate(eig_pairs):
    print(f'eigenvalue {i+1}: {(j[0]/eigv_sum).real:.2%}')

W = np.concatenate([eig_pairs[i][1].reshape(num_descriptor,1) for i in range(num_eigenvector)], axis=1)
print('Matrix W:\n', W.real)

### STEP 8. Transform the samples onto the new subspace
W = W.real
X_lda = X.dot(W)
y = np.concatenate([np.zeros(nDataPoints),np.ones(nDataPoints),np.ones(nDataPoints)+1])

np.savetxt("FLDA.csv", X_lda, delimiter=",", fmt="%.6f", header="LD1, LD2", comments="")
df2 = pd.read_csv('FLDA.csv')
df2['class'] = y
df2.to_csv('FLDA.csv', encoding='utf-8', index=False)

Eigenvalues in decreasing order:

20.488865937510138
5.397305314980233
2.460044647123976e-15
7.602574054109184e-16
7.602574054109184e-16
2.936270329955646e-16
5.926141028976611e-17
Variance explained:

eigenvalue 1: 79.15%
eigenvalue 2: 20.85%
eigenvalue 3: 0.00%
eigenvalue 4: 0.00%
eigenvalue 5: 0.00%
eigenvalue 6: 0.00%
eigenvalue 7: 0.00%
Matrix W:
 [[-0.42   -0.2525]
 [ 0.4925 -0.2693]
 [-0.3204  0.396 ]
 [-0.028   0.5397]
 [ 0.522   0.3364]
 [ 0.2858  0.2591]
 [ 0.3515 -0.485 ]]
